# DSPy Learning

## Initialization

In [5]:
from dotenv import load_dotenv
import os
import dspy

/Users/mayanksingh/.pyenv/versions/3.13.0/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
llm = dspy.LM('openai/gpt-4o', api_key=os.getenv("OPENAI_API_KEY"))
dspy.configure(lm=llm)

## 1. Language models

https://dspy.ai/learn/programming/language_models/

### 1.1 Calling LLM directly

In [11]:
llm("What is the meaning of life? One integer answer", temperature=0.9)

['42']

In [12]:
llm(messages=[{
    "role": "system", "content": "Give integer answers only",
    "role": "user", "content": "What is the meaning of life?"
    }])

['The meaning of life is a question that has been pondered by philosophers, theologians, scientists, and thinkers for centuries, and it often depends on individual beliefs, cultural perspectives, and personal experiences. Some people find meaning through religion or spirituality, believing that life has a divine purpose. Others find meaning through relationships, personal achievements, or contributing to the well-being of others. Some philosophical perspectives suggest that life has no inherent meaning, and it is up to each individual to create their own purpose. Ultimately, the meaning of life is a deeply personal and subjective question, and different people may arrive at different answers based on their own values and experiences.']

### 1.2 Using LM with DSPy modules

In [33]:
# Define a module (CoT), assign it a signature (return an answer, given a question)
cot_qa_module = dspy.ChainOfThought('question -> answer')
# cot_qa_module = dspy.ChainOfThought('q -> a') # can name it anything, doesn't have to be 'question -> answer'
cot_qa_module

self = Predict(StringSignature(question -> reasoning, answer
    instructions='Given the fields `question`, produce the fields `answer`.'
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Question:', 'desc': '${question}'})
    reasoning = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${reasoning}', '__dspy_field_type': 'output'})
    answer = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 'Answer:', 'desc': '${answer}'})
))

In [35]:
response = cot_qa_module(question="What is the meaning of life?")
# response = cot_qa_module(q="What is the meaning of life?")
response

Prediction(
    reasoning='The meaning of life is a philosophical question that has been contemplated by humans for centuries. Different cultures, religions, and individuals have proposed various interpretations. Some suggest that the meaning of life is to seek happiness, fulfillment, and personal growth, while others believe it is about serving a higher purpose or contributing to the well-being of others. Ultimately, the meaning of life may vary from person to person, shaped by personal experiences, beliefs, and values.',
    answer="The meaning of life is subjective and can vary greatly among individuals. It may involve seeking happiness, personal growth, and contributing to the well-being of others, or it may be defined by one's beliefs and experiences."
)

The response has a reasoning component and the final answer from the CoT module

### 1.3 Using multiple LLMs

In [37]:
dspy.configure(lm=dspy.LM('openai/gpt-4o-mini'))
response = cot_qa_module(question="What is the meaning of life?")
print('GPT-4o-mini:', response.answer)

with dspy.context(lm=dspy.LM('openai/gpt-3.5-turbo')):
    response = cot_qa_module(question="What is the meaning of life?")
    print('GPT-3.5-turbo:', response.answer)

GPT-4o-mini: The meaning of life is subjective and can vary greatly among individuals. It may involve seeking happiness, personal growth, and contributing to the well-being of others, or it may be defined by one's beliefs and experiences.
GPT-3.5-turbo: The meaning of life is a deeply personal and subjective concept that varies from person to person. Some find meaning through relationships, others through accomplishments, spirituality, or personal growth. Ultimately, the meaning of life is a question that each individual must explore and define for themselves.


### 1.4 Configure LM generation

By default LMs are cached in DSPy, but you can disable this feature

stop: stop token

In [47]:
gpt_4o_mini = dspy.LM('openai/gpt-4o-mini', temperature=0.9, max_tokens=3000, stop=".", cache=False)

In [48]:
gpt_4o_mini("What is the meaning of life?")

['The meaning of life is a profound philosophical question that has been explored by thinkers, theologians, and scientists throughout history']

In [50]:

gpt_4o_mini("What is the meaning of life? One integer answer only")

['42']

Each LLM maintains history of interactions, along with the token input and output and also cost

In [67]:
for record in gpt_4o_mini.history:
    print(f"Input: {record['prompt']}")
    print(f"Output: {record['outputs'][-1]}")
    print(f"Cost: {record['cost']}")

Input: What is the meaning of life?
Output: The meaning of life is a profound philosophical question that has been explored by thinkers, theologians, and scientists throughout history
Cost: 1.59e-05
Input: What is the meaning of life? One integer answer only
Output: 42
Cost: 3.2999999999999997e-06
